In [1]:
from ExampleAnalyzer.main import ExampleAnalyzer
from framework.AnalyzerUtilities import *

In [2]:
ana = ExampleAnalyzer()

--- load_feature_config from .csv files ---
load inFeatures csv:  /home/zchen/Documents/Analysis/nanoblt/ExampleAnalyzer/featureConfig/inFeature.csv
load internalFeatures csv:  /home/zchen/Documents/Analysis/nanoblt/ExampleAnalyzer/featureConfig/internalFeature.csv
load outFeatures csv:  /home/zchen/Documents/Analysis/nanoblt/ExampleAnalyzer/featureConfig/outFeature.csv
Successfully load all .csv files
--- lgenerate_cuda_struct_declaration ---
generate cuda struct 'MaskEvents' 
generate cuda struct 'Events' 
generate cuda struct 'EventsInternal' 
generate cuda struct 'EventsOut' 
Successfully generate all cuda struct 
--- compile_cuda_kernels in .cu files ---
Successfully complie cuda kernels


In [3]:
#infile = '/home/zchen/cms/data/EGamma_Run2018B/E4BC7705-C2F2-1D4D-BBD5-3E375685BA51.root'
infile = '../data/tt.root'
ana.process_infile(infile,'example.h5')

time mask 0.17443084716796875
pass mask 364876/855000, eff=0.427
time init_events read tree 2.8450074195861816
time init_events apply mask 0.19158673286437988
time init_events flat jaggedArray 0.17490720748901367
time init_events bool2int 0.0008370876312255859
time init_events cumsum 0.0016031265258789062
time init_events device soa 8.106231689453125e-05
time init_events internal-out 0.0024971961975097656
this is event selection
time totalGPU 0.014783382415771484
this is post process
eventsOut is  {'nPassElectron': array([2, 1, 1, ..., 1, 1, 0], dtype=int32), 'nPassMuon': array([0, 2, 1, ..., 1, 0, 0], dtype=int32)}
this is store data example.h5


In [4]:
ana.tree = uproot.open('../data/tt.root')["Events"]   
ana.get_mask()
ana.init_events()

pass mask 364876/855000, eff=0.427


In [5]:
ana.copy_events_to_gpu()
ana.object_selection()
ana.event_selection()
ana.copy_events_from_gpu()
#ana.clear_devents()
ana.postprocess()
ana.eventsOut

this is event selection
this is post process


{'nPassElectron': array([2, 1, 1, ..., 1, 1, 0], dtype=int32),
 'nPassMuon': array([0, 2, 1, ..., 1, 0, 0], dtype=int32)}

In [8]:
ana.deventsInternal.copy_from_gpu()

In [11]:
ana.eventsInternal.iPassElectron.reshape(-1,4)

array([[0, 1, 0, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int32)

In [8]:
np.array()

NameError: name 'np' is not defined

In [5]:
ana.store("s")
ana.clear_events()

In [3]:
ana.process_infile('data/tt.root','s')
'/home/zchen/cms/data/EGamma_Run2018B/E4BC7705-C2F2-1D4D-BBD5-3E375685BA51.root'

pass mask 364876/855000, eff=0.427
this is event selection
this is post process
eventsOut is  {'nPassElectron': array([2, 1, 1, ..., 1, 1, 0], dtype=int32), 'nPassMuon': array([0, 2, 1, ..., 1, 0, 0], dtype=int32)}
this is store data s


In [6]:
ana.events.nev

364876

In [4]:
inputDir  = "/home/zchen/cms/data/EGamma_Run2018B/"
fileNames = glob.glob(inputDir+"*.root")
fileName = fileNames[1]

In [5]:
fileName

'/home/zchen/cms/data/EGamma_Run2018B/E4BC7705-C2F2-1D4D-BBD5-3E375685BA51.root'

In [ ]:
def compile_cuda_kernels(self):
    cufiles = ['./ExampleAnalyzer.cu']
    self.knl_mask = cu.elementwise.ElementwiseKernel(
        arguments="bool *mask, bool *HLT_Ele32_WPTight_Gsf, bool *HLT_IsoMu24, int *nElectron, int *nMuon",
        operation="mask[i] = (HLT_Ele32_WPTight_Gsf[i] || HLT_IsoMu24[i]) && (nElectron[i]>=2 || nMuon[i]>=2)" )

    # read code from src files
    code = self.codeMaskEventsStruct + self.codeEventsStruct
    for cufile in cufiles:
        f = open(cufile,"r") 
        code += f.read()
        f.close()

    # complie cuda code
    module = cu.compiler.SourceModule(  code )
    self.knl_objectSelection_electron = module.get_function('knl_objectSelection_electron')
    self.knl_objectSelection_muon     = module.get_function('knl_objectSelection_muon')